# Working with Binance API
- https://python-binance.readthedocs.io/en/latest/overview.html

In [1]:
import datetime as dt
import os
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import dateparser

import math
import matplotlib.pyplot as plt
from binance.client import Client
from tqdm import tqdm

from trading.websockets.binance import listener
from trading.database import data_point, utils

In [2]:
DB_NAME = "trading_data"
utils.connect_to_db(DB_NAME)

In [3]:
col = data_point.DataPoint._get_collection()

In [4]:
ethbtc_trades = col.find({"symbol":"ETHBTC", "event_type": "trade"})

In [6]:
trade_data = []
for i in tqdm(ethbtc_trades):
    trade_data.append(i["data"])

147068it [00:07, 18882.87it/s]


# Use this if possible
https://www.groundai.com/project/neural-jump-stochastic-differential-equations/1

In [7]:
pd.DataFrame(trade_data)

,trade_id,price,quantity,buy_order_id,sell_order_id,trade_time,limit_buy
0,212101908,0.03232400,1.56700000,1192761289,1192761271,2021-01-05 16:07:39.583,False
1,212101909,0.03232400,0.43300000,1192761289,1192761279,2021-01-05 16:07:39.583,False
2,212101910,0.03232300,0.23000000,1192760758,1192761297,2021-01-05 16:07:39.882,True
3,212101911,0.03232500,7.67400000,1192761318,1192761268,2021-01-05 16:07:40.413,False
4,212101912,0.03232400,0.42300000,1192761328,1192761322,2021-01-05 16:07:40.719,False
...,...,...,...,...,...,...,...
147063,212248971,0.03167800,0.20800000,1194135838,1194135810,2021-01-05 23:39:25.949,False
147064,212248972,0.03168100,4.49400000,1194135847,1194135788,2021-01-05 23:39:26.056,False
147065,212248973,0.03167900,0.23700000,1194135850,1194135852,2021-01-05 23:39:26.145,True
147066,212248974,0.03167900,0.23700000,1194135850,1194135853,2021-01-05 23:39:26.157,True


In [ ]:
col.index_information()

In [ ]:
agg = [
    {
        "$group": {
            "_id": "$symbol",
            "count": {"$sum": 1}
        }
    }
]
res = col.aggregate(agg)

In [ ]:
list(res)